In [1]:
from keras.applications import vgg19
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt

C:\Users\IS96273\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
content_image_path = "C:/Users/IS96273/Desktop/content.jpeg"
style_image_path = "C:/Users/IS96273/Desktop/style.jpg"

height = 224; weight = 224 #original size of vgg

In [3]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(height, weight))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0) #layer block1_conv1: expected ndim=4, found ndim=3
    img = preprocess_input(img)
    return img

In [4]:
content_image = preprocess_image(content_image_path)
style_image = preprocess_image(style_image_path)

In [5]:
random_pixels = np.random.randint(256, size=(height, weight, 3))
generated_image = preprocess_input(np.expand_dims(random_pixels, axis=0))

In [6]:
print(content_image.shape)
print(style_image.shape)
print(generated_image.shape)

(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)


In [7]:
content_model = vgg19.VGG19(input_tensor=K.variable(content_image), weights='imagenet')
style_model = vgg19.VGG19(input_tensor=K.variable(content_image), weights='imagenet')
generated_model = vgg19.VGG19(input_tensor=K.variable(generated_image), weights='imagenet')

In [8]:
content_outputs = dict([(layer.name, layer.output) for layer in content_model.layers])
style_outputs = dict([(layer.name, layer.output) for layer in style_model.layers])
generated_outputs = dict([(layer.name, layer.output) for layer in generated_model.layers])

In [9]:
def content_loss(content, generated):
    return K.sum(K.square(content - generated))

content_features = content_outputs['block5_conv2']
generated_features = generated_outputs['block5_conv2']

contentloss = content_loss(content_features, generated_features)

In [10]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, generated):
    style_gram = gram_matrix(style)
    content_gram = gram_matrix(generated)
    channels = 3
    size = height * weight
    return K.sum(K.square(style_gram - content_gram)) / (4. * (pow(channels,2)) * (pow(size,2)))

feature_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

styleloss = K.variable(0)
for layer_name in feature_layers:
    style_features = style_outputs[layer_name]
    generated_features = generated_outputs[layer_name]
    styleloss = styleloss + style_loss(style_features[0], generated_features[0])

In [11]:
alpha = 0.025; beta = 0.2
total_loss = alpha * contentloss + beta * styleloss

In [13]:
#gradients = K.gradients(loss, generated_model.trainable_weights)
gradients = K.gradients(total_loss, generated_model.input)

Tensor("gradients/block1_conv1_2/convolution_grad/Conv2DBackpropInput:0", shape=(1, 224, 224, 3), dtype=float32)


In [19]:
print(gradients[0])
print(generated_image.shape)

Tensor("gradients/block1_conv1_2/convolution_grad/Conv2DBackpropInput:0", shape=(1, 224, 224, 3), dtype=float32)
(1, 224, 224, 3)


In [ ]:
learning_rate = np.array([0.1])
#generated_image = generated_image - learning_rate * gradients